## Backtesting Dual moving average strategy on multiple assets using vectorbt

Reference: # http://qubitquants.pro/multi_asset_portfolio_simulation/index.html


In this tutorial, we will talk about Multi Asset Portfolio Simulation, beginning with:

Running Multi-asset Portfolio Backtesting simulation using vbt.Portfolio.from_signals() like:
1. Unified Portfolio Simulation
2. Asset-wise Discrete Portfolio Simulation
3. Grouped Portfolio Simulation

## 1) Unified Portfolio Simulation

In [2]:
import vectorbt as vbt
import numpy
import pandas
from plotly.offline import init_notebook_mode, iplot

import warnings

warnings.filterwarnings("ignore")

init_notebook_mode(connected=True)

In [3]:
symbols = ["MSFT","AAPL","GOOGL"]
close_price = vbt.YFData.download(symbols, interval="1d",
                        missing_index="drop",
                        start="2020-01-01").get("Close")
print(close_price)

symbol                           MSFT        AAPL       GOOGL
Date                                                         
2019-12-31 05:00:00+00:00  155.329636   72.552094   66.969498
2020-01-02 05:00:00+00:00  158.205765   74.207466   68.433998
2020-01-03 05:00:00+00:00  156.235825   73.486023   68.075996
2020-01-06 05:00:00+00:00  156.639694   74.071579   69.890503
2020-01-07 05:00:00+00:00  155.211456   73.723213   69.755501
...                               ...         ...         ...
2023-07-31 04:00:00+00:00  335.920013  196.449997  132.720001
2023-08-01 04:00:00+00:00  336.339996  195.610001  131.550003
2023-08-02 04:00:00+00:00  327.500000  192.580002  128.380005
2023-08-03 04:00:00+00:00  326.660004  191.169998  128.449997
2023-08-04 04:00:00+00:00  327.779999  181.990005  128.110001

[905 rows x 3 columns]


In [4]:
"""
Setup entry and exit condition, which is with moving average (MA) 
crossover combination of fast MA of 9 days and slow MA of 17 days
"""

SMA_9 = vbt.MA.run(close_price, window=9)
SMA_17 = vbt.MA.run(close_price, window=17)

entries = SMA_9.ma_crossed_above(SMA_17)
exits = SMA_9.ma_crossed_below(SMA_17)

##### Description of a few Parameter settings for vbt.Portfolio.from_signals()

We will see a short description of the new parameters of vbt.Portfolio.from_signals() 

a.) `size` : Specifies the position size in units. For any fixed size, you can set to any number to buy/sell some fixed amount or value. For any target size, you can set to any number to buy/sell an amount relative to the current position or value. If you set this to np.nan or 0 it will get skipped (or close the current position in the case of setting 0 for any target size). Set to `np.inf` to buy for all cash, or `-np.inf` to sell for all free cash. A point to remember setting to `np.inf` may cause the scenario for the portfolio simulation to become heavily weighted to one single instrument. So use a sensible size related.

b.) `size_type`: Choose units to be used for the `size`. In this tutorial, we use `percent` and for other parameter like `amount`, `value`, `TargetValue`,`TargetAmount`, please refer here: https://vectorbt.dev/api/portfolio/enums/#vectorbt.portfolio.enums.SizeType for more explanation.

b.) `init_cash` : Initial capital per column (or per group with cash sharing). By setting it to auto the initial capital is automatically decided based on the position size you specify in the above size parameter.

c.) `cash_sharing` : Accepts a boolean (`True` or `False`) value to specify whether cash sharing is to be disabled or if enabled then cash is shared across all the assets in the portfolio or cash is shared within the same group.
If `group_by` is None and `cash_sharing` is True, group_by becomes True to form a single group with cash sharing. Example:
Consider three columns (3 assets), each having $100 of starting capital. If we built one group of two columns and one group of one column, the `init_cash` would be np.array([200, 100]) with cash sharing enabled and np.array([100, 100, 100]) without cash sharing.

d.) `call_seq` : Default sequence of calls per row and group. Controls the sequence in which order_func_nb is executed within each segment. For more details of this function kindly refer the documentation.

e.) `group_by` : can be boolean, integer, string, or sequence to call multi-level indexing and can accept both level names and level positions. In this tutorial I will be setting group_by = True to treat the entire portfolio simulation in a unified manner for all assets in congruence with cash_sharing = True. When I want to create custom groups with specific symbols in each group then I will be setting group_by = 0 to specify the level position (in multi-index levels) as the first in the hierarchy.``

In [5]:
"""In this section, we run the portfolio simulation treating the entire portfolio as a singular asset by enabling the following parameters in the pf.from_signals():

cash_sharing = True
group_by = True
call_seq = "auto"
size = 1000
"""
unified_portfolio = vbt.Portfolio.from_signals(close_price, 
                           entries,
                           exits,
                           init_cash=100000, # in $
                           fees=0.0025, # in %
                           slippage=0.0025, # in %
                           freq="1D",
                           direction="LongOnly",
                           group_by=True,
                           cash_sharing=True,
                           call_seq="auto",
                           size_type="value",
                           size=10000) # 

unified_portfolio.stats()

Start                          2019-12-31 05:00:00+00:00
End                            2023-08-04 04:00:00+00:00
Period                                 905 days 00:00:00
Start Value                                     100000.0
End Value                                   114588.71509
Total Return [%]                               14.588715
Benchmark Return [%]                          117.719565
Max Gross Exposure [%]                         37.042728
Total Fees Paid                              3799.697909
Max Drawdown [%]                                 9.16551
Max Drawdown Duration                  410 days 00:00:00
Total Trades                                          76
Total Closed Trades                                   74
Total Open Trades                                      2
Open Trade PnL                               2209.124536
Win Rate [%]                                   41.891892
Best Trade [%]                                 53.523205
Worst Trade [%]                

In [6]:
unified_portfolio.plot(group_by=True, subplots=["cum_returns","cash","value"]).show(render="svg")

### Asset-wise Discrete Portfolio Simulation

In this section, we will see how to run the portfolio simulation for each asset in the portfolio independently. 

In [7]:
discrete_portfolio = vbt.Portfolio.from_signals(close_price, 
                           entries,
                           exits,
                           init_cash=100000, # in $
                           fees=0.0025, # in %
                           slippage=0.0025, # in %
                           direction="LongOnly",
                           freq="1D",
                           group_by=False,
                           call_seq="auto",
                           size_type="value",
                           size=10000) # For each trades, limit the position size in $


In [8]:
# trade each assets with start value of 100,000 and apply the trading strategy respectively to see which compa

stats_df = pandas.concat([unified_portfolio.stats()] + 
                         [discrete_portfolio[symbol].stats() for symbol 
                          in discrete_portfolio.wrapper.columns], axis = 1)
stats_df.rename(inplace = True, columns = {'group':'unified portfolio'})  
stats_df

,unified portfolio,"(9, 17, MSFT)","(9, 17, AAPL)","(9, 17, GOOGL)"
Start,2019-12-31 05:00:00+00:00,2019-12-31 05:00:00+00:00,2019-12-31 05:00:00+00:00,2019-12-31 05:00:00+00:00
End,2023-08-04 04:00:00+00:00,2023-08-04 04:00:00+00:00,2023-08-04 04:00:00+00:00,2023-08-04 04:00:00+00:00
Period,905 days 00:00:00,905 days 00:00:00,905 days 00:00:00,905 days 00:00:00
Start Value,100000.0,100000.0,100000.0,100000.0
End Value,114588.71509,100738.838304,109214.610669,104635.266117
Total Return [%],14.588715,0.738838,9.214611,4.635266
Benchmark Return [%],117.719565,111.02219,150.840461,91.296045
Max Gross Exposure [%],37.042728,13.156695,15.374068,13.08177
Total Fees Paid,3799.697909,1408.869269,1048.044713,1342.783926
Max Drawdown [%],9.16551,4.972317,2.423974,4.499051


-------------------------------------------------

#### (Optional) Additional work - just to check trading orders & PnL of AAPL

In [17]:
discrete_portfolio[(9, 17, "AAPL")].plot().show(render="svg")